In [1]:
import pandas as pd
from rdkit import Chem
import numpy as np
import matplotlib.pyplot as plt
import multiprocessing as mp

from tqdm.notebook import tqdm

from tqdm.contrib.concurrent import process_map

In [2]:
data = pd.read_csv("./data_files/smiles_min_dist_natoms.csv",index_col=0)

In [6]:
drugs = Chem.SDMolSupplier("./drugbank_structures.sdf")
drugs = [i for i in drugs if i]
for d in drugs:
    Chem.SanitizeMol(d)
#     Chem.Kekulize(d,clearAromaticFlags=True)

In [7]:
# the one just for number of lines - used for the entire drugbank
def rowfunc(smiles):
    # search the smiles in "row" through all the structures in "database"
    substruct = Chem.MolFromSmiles(smiles)
    Chem.SanitizeMol(substruct)
#     Chem.Kekulize(substruct,clearAromaticFlags=True)
    
    substruct_matches = 0
    for database_mol in tqdm(drugs):
        if database_mol.HasSubstructMatch(substruct):
            substruct_matches += 1
    return substruct_matches

In [ ]:

result = process_map(rowfunc, list(data.smiles), chunksize=1000)


  0%|          | 0/222747 [00:00<?, ?it/s]

In [ ]:
data["drugbank_matches"] = result

In [ ]:
data.sort_values("drugbank_matches")

In [ ]:
data

In [ ]:
# can save to csv as a backup 
# data.to_csv("./data_files/acid_amine_smiles_mindist_dbank.csv")

## search within the covid-19 antivirals
formatted for chord diagram


In [4]:
data = pd.read_csv("./data_files/acid_amine_smiles_mindist_dbank.csv",index_col=0)

In [5]:
data1 = data.sort_values("min_dist_all")

In [7]:
match_smiles = list(data1.smiles)

In [8]:
CA_smiles = list(pd.read_csv("./data_files/covid_antiviral_smiles.csv").smiles)

In [9]:
CA_mols = [Chem.MolFromSmiles(s) for s in CA_smiles]
for m in CA_mols:
    Chem.SanitizeMol(m)
#     Chem.Kekulize(m,clearAromaticFlags=False)

In [ ]:
# comprehensive one for circos plots
def get_matches_all(database,matcher,output_filename):
    results = []
    out_file = open(output_filename,"w")
    out_file.write("ndrug,nsub,nmatches,smiles\n")
    # loop through database (the smiles of the drugs, etc that you want to match with)

    for i_d, d in enumerate(tqdm(database)):
        
        database_mol = d
        Chem.SanitizeMol(database_mol)
#         Chem.Kekulize(database_mol,clearAromaticFlags=True)
        
        # loop through the set of smiles to match (enumerated substructs)
        for i_r, r in enumerate(tqdm(matcher.itertuples())):
            sub = Chem.MolFromSmiles(r.smiles)
            Chem.SanitizeMol(sub)
#             Chem.Kekulize(sub,clearAromaticFlags=True)
            matches = database_mol.GetSubstructMatches(sub)
            if matches: 
                out_file.write(f"{i_d},{i_r},{len(matches)},{r.smiles}\n")
    out_file.close()
    return "done"

In [ ]:
get_matches_all(CA_mols,data1,"./data_files/C19av_matches_by_mindist.csv")